In [1]:
#Bring in datasets

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
from scipy.stats import linregress
import os
import csv
from sqlalchemy import create_engine

In [2]:
# Create happiness dataframe
happiness_df = pd.read_csv('2017_happiness_index.csv')
happiness_df.drop_duplicates()
happiness_df

,country,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182
...,...,...,...,...,...,...,...,...,...,...,...,...
147,Rwanda,151,3.471,3.543030,3.398970,0.368746,0.945707,0.326425,0.581844,0.252756,0.455220,0.540061
148,Syria,152,3.462,3.663669,3.260331,0.777153,0.396103,0.500533,0.081539,0.493664,0.151347,1.061574
149,Tanzania,153,3.349,3.461430,3.236570,0.511136,1.041990,0.364509,0.390018,0.354256,0.066035,0.621130
150,Burundi,154,2.905,3.074690,2.735310,0.091623,0.629794,0.151611,0.059901,0.204435,0.084148,1.683024


In [3]:
# Create water dataframe
water_df = pd.read_csv('water_index.csv')
water_df.drop_duplicates()
water_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,0.278,0.278,0.299,0.320,0.341,0.363,0.384,0.408,0.433,0.458,0.484,0.509,0.535,0.562,0.588,0.615,0.643,0.671
1,Albania,0.879,0.879,0.879,0.879,0.879,0.879,0.879,0.879,0.879,0.879,0.878,0.878,0.879,0.895,0.910,0.910,0.910,0.910
2,Algeria,0.898,0.901,0.904,0.907,0.909,0.912,0.914,0.917,0.919,0.922,0.924,0.926,0.928,0.931,0.933,0.935,0.935,0.936
3,Andorra,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
4,Angola,0.411,0.423,0.434,0.444,0.454,0.463,0.472,0.480,0.488,0.496,0.504,0.512,0.520,0.528,0.535,0.543,0.551,0.558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Vietnam,0.801,0.810,0.819,0.828,0.836,0.845,0.853,0.862,0.870,0.878,0.887,0.895,0.903,0.910,0.923,0.932,0.940,0.947
191,Yemen,0.378,0.381,0.383,0.403,0.423,0.443,0.463,0.482,0.502,0.521,0.540,0.559,0.577,0.596,0.614,0.632,0.633,0.635
192,Zambia,0.495,0.500,0.507,0.514,0.521,0.527,0.533,0.540,0.546,0.552,0.558,0.564,0.570,0.576,0.581,0.587,0.592,0.600
193,Zimbabwe,0.717,0.719,0.716,0.711,0.706,0.701,0.696,0.690,0.685,0.680,0.675,0.670,0.665,0.660,0.655,0.650,0.645,0.641


In [4]:
#Clean water data to years 2015, 2016, 2017
clean_water_df=water_df.drop(columns=["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2008", "2009", "2010", "2011", "2012", "2013", "2014"])
clean_water_df

,country,2015,2016,2017
0,Afghanistan,0.615,0.643,0.671
1,Albania,0.910,0.910,0.910
2,Algeria,0.935,0.935,0.936
3,Andorra,1.000,1.000,1.000
4,Angola,0.543,0.551,0.558
...,...,...,...,...
190,Vietnam,0.932,0.940,0.947
191,Yemen,0.632,0.633,0.635
192,Zambia,0.587,0.592,0.600
193,Zimbabwe,0.650,0.645,0.641


In [5]:
#Create 1 clean data set

In [6]:
merge1_df = pd.merge(happiness_df, clean_water_df, on="country", how="left")
merge1_df.dropna()
merge1_df.drop_duplicates()
merge1_df

,country,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual,2015,2016,2017
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,1.000,1.000,1.000
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,1.000,1.000,1.000
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715,1.000,1.000,1.000
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716,1.000,1.000,1.000
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182,1.000,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Rwanda,151,3.471,3.543030,3.398970,0.368746,0.945707,0.326425,0.581844,0.252756,0.455220,0.540061,0.563,0.570,0.577
148,Syria,152,3.462,3.663669,3.260331,0.777153,0.396103,0.500533,0.081539,0.493664,0.151347,1.061574,0.969,0.971,0.972
149,Tanzania,153,3.349,3.461430,3.236570,0.511136,1.041990,0.364509,0.390018,0.354256,0.066035,0.621130,0.530,0.548,0.567
150,Burundi,154,2.905,3.074690,2.735310,0.091623,0.629794,0.151611,0.059901,0.204435,0.084148,1.683024,0.596,0.602,0.608


In [7]:
merge1_df = merge1_df.rename(columns={"2015": "Water_Index_2015", "2016": "Water_Index_2016", "2017": "Water_Index_2017"})
merge1_df

,country,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual,Water_Index_2015,Water_Index_2016,Water_Index_2017
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,1.000,1.000,1.000
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,1.000,1.000,1.000
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715,1.000,1.000,1.000
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716,1.000,1.000,1.000
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182,1.000,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Rwanda,151,3.471,3.543030,3.398970,0.368746,0.945707,0.326425,0.581844,0.252756,0.455220,0.540061,0.563,0.570,0.577
148,Syria,152,3.462,3.663669,3.260331,0.777153,0.396103,0.500533,0.081539,0.493664,0.151347,1.061574,0.969,0.971,0.972
149,Tanzania,153,3.349,3.461430,3.236570,0.511136,1.041990,0.364509,0.390018,0.354256,0.066035,0.621130,0.530,0.548,0.567
150,Burundi,154,2.905,3.074690,2.735310,0.091623,0.629794,0.151611,0.059901,0.204435,0.084148,1.683024,0.596,0.602,0.608


In [8]:
is_NaN = merge1_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = merge1_df[row_has_NaN]
print(rows_with_NaN)

Empty DataFrame
Columns: [country, Happiness_Rank, Happiness_Score, Whisker_high, Whisker_low, Economy_GDP_per_Capita, Family, Health_Life_Expectancy, Freedom, Generosity, Trust_Government_Corruption, Dystopia_Residual, Water_Index_2015, Water_Index_2016, Water_Index_2017]
Index: []


In [9]:
rds_connection_string = "postgres:PostgreSQL@localhost:5432/happy_water"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
engine.table_names()

['happy_region', 'happy_water']

In [12]:
merge1_df.to_sql(name='happy_water', con=engine, if_exists='append', index=False)

In [13]:
pd.read_sql_query('select * from happy_water', con=engine).head()

,country,Happiness_Rank,Happiness_Score,Whisker_high,Whisker_low,Economy_GDP_per_Capita,Family,Health_Life_Expectancy,Freedom,Generosity,Trust_Government_Corruption,Dystopia_Residual,Water_Index_2015,Water_Index_2016,Water_Index_2017
0,Norway,1,8,8,7,2,2,1,1,0,0,2,1,1,1
1,Denmark,2,8,8,7,1,2,1,1,0,0,2,1,1,1
2,Iceland,3,8,8,7,1,2,1,1,0,0,2,1,1,1
3,Switzerland,4,7,8,7,2,2,1,1,0,0,2,1,1,1
4,Finland,5,7,8,7,1,2,1,1,0,0,2,1,1,1


In [14]:
pd.read_sql_query('select * from happy_region', con=engine).head()

,country,Region
0,Denmark,Western Europe
1,Switzerland,Western Europe
2,Iceland,Western Europe
3,Norway,Western Europe
4,Finland,Western Europe
